In [ ]:
%matplotlib qt
from threading import Thread
from random import randint
from time import sleep, time
from bluesky import RunEngine
from bluesky.callbacks import LiveGrid
from bluesky.plans import grid_scan
from bluesky.plan_stubs import kickoff, complete, collect, \
    open_run, close_run, mv, one_nd_step, trigger_and_read, move_per_step, \
    create, save
from ophyd import Device, ADComponent, Component
from ophyd.flyers import FlyerInterface
from ophyd.status import AndStatus, SubscriptionStatus
from ophyd.sim import motor1, motor2, rand, det4, FakeEpicsSignalWithRBV, \
    FakeEpicsSignal, FakeEpicsSignalRO
from matplotlib.pyplot import ion
from bluesky.utils import install_nb_kicker
install_nb_kicker()

ion()
RE = RunEngine()

## Fly Scan

In [ ]:
class SimAreaDetectorAcquire(Device):
    det = ADComponent(FakeEpicsSignalWithRBV, "Value")
    acquire = ADComponent(FakeEpicsSignalWithRBV, "Acquire")
    capture = ADComponent(FakeEpicsSignalWithRBV, "Capture")
    data_array = []

area_detector_cam = SimAreaDetectorAcquire("SIM:", name="area_detector_cam")

In [ ]:
class SimTatu(Device):
    activate = Component(FakeEpicsSignalWithRBV, "Activate")
    steps = Component(FakeEpicsSignalWithRBV, "Steps")
    
tatu = SimTatu("SIM:", name="tatu")

In [ ]:
def fly_scan_simulation(tatu, detector):
    step = tatu.steps.get()
    while step < 5:
        new_step = tatu.steps.get() + 1
        tatu.steps.set(new_step).wait()
        det_val = randint(-10, 10)
        area_detector_cam.data_array.append({
            'time': time(),
            'data': {'det': det_val}, 
            'timestamps': {'det': time()}
        })
        
        sleep(1)
        step = new_step

In [ ]:
class FlyerDevice(FlyerInterface, Device):
    
    def kickoff(self):
        print("Start asynchronous task")
        area_detector_cam.capture.set(1).wait()
        area_detector_cam.acquire.set(1).wait()
        tatu.steps.set(0).wait()
        
        fly_scan_thread = Thread(
            target=fly_scan_simulation, kwargs={"tatu": tatu, "detector": area_detector_cam})
        fly_scan_thread.start()
        
        return tatu.activate.set(1)
            

    def tatu_monitor(self, **kwargs):
        tatu_steps = kwargs['obj']
        curr_step = tatu_steps.get()
        print(f"Monitoring: {curr_step}")
        
        if curr_step == 5:
            return True
        return False

    def complete(self):
        print("Waiting for fly scan to complete")
        return SubscriptionStatus(tatu.steps, callback=self.tatu_monitor)
    
    def describe_collect(self):
        return {"decription": {}}
    
    def collect(self):
        print("Collecting data after scan")
        return [{"data": {}, "timestamps": {}, "time": 0, "seq_num": 0}]

In [ ]:
def flyscan(flyer):
    yield from open_run()
    yield from kickoff(flyer, wait=True)
    yield from complete(flyer, wait=True)
    yield from close_run()

flyer_device = FlyerDevice(name="flyer_device")
RE(flyscan(flyer_device))

## per_step

In [ ]:
def custom_step(detectors, step, pos_cache):
    print("--------------------------")
    print(f"Grid step: {step.values()}")
    print("Before plan step")
    yield from one_nd_step(detectors, step, pos_cache)
    print("After plan step")

In [ ]:
RE(
    grid_scan([det4], motor1, -1, 1, 5, motor2, -2, 2, 5, per_step=custom_step),
    LiveGrid((5, 5), 'det4'))

- trigger_and_read: Triggera e lê uma lista de detectores e motores e armazena seus valores.
- move_per_step: Envia setpoints para vários dispositivos que permitem ser setados e espera eles chegaram na posição.
- one_shot: Step para o plano 'count'
- one_1d_step: Step para planos com 1 Dimensão
- one_nd_step: Step para planos com N Dimensões

In [ ]:
def one_shot(detectors, motor, step):
    yield from trigger_and_read(list(detectors) + [motor])

In [ ]:
def one_1d_step(detectors, motor, step):
    yield from move()
    yield from trigger_and_read(list(detectors) + [motor])

In [ ]:
def one_nd_step(detectors, step, pos_cache):
    yield from move_per_step(step, pos_cache)
    yield from trigger_and_read(list(detectors) + list(motors))